# example how to extract the S1/S2 features (maybe additional ECDC features) in openEO
In this example we look on each step to generate a ML usable training dataset from given reference points.

1. load reference points & convert to EPSG:4326 if needed
2. add the GeoIdentifier for 10m patch, the patch center coordinate in EPSG:4326 and other important attributes
3. create the job dataframe by grouping the reference points by UTM zones or UTM 10K tiles
4. add all needed columns to the job dataframe to create a start_function for the opneEO MultiBackendJobManager
5. write start_function for JobMultiBackend Manager
6. setup the client-side processing with automatic retry and credit cost-evaluation
7. execute
8. training data check and visualization